## Introduction to Altair (part 2)

In this section, we will focus on including the interaction options in the Altair. 

Through interaction we can transform static images into tools for exploration: highlighting points of interest, zooming in to reveal finer-grained patterns, and linking across multiple views to reason about multi-dimensional relationships.

We will visualize the same car data in part 1 from the vega-datasets collection. 

In [2]:
# Install the module
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'

### Set parameters

We'll start with a scatter plot of horsepower versus miles per gallon, with a color encoding for the origin. We set the opacity as 0.5. 

In [4]:
alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.Color("Origin:N"),
    opacity = alt.OpacityValue(0.5)
)

alt.Chart(...)

The first thing we can do is to add a slide bar to change the settings in the figure, like the opacity. 

In [6]:
slider = alt.binding_range(min = 0, max = 1, step = 0.05, name = "Opacity:")
op_var = alt.param(value = 0.1, bind = slider)

alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.Color("Origin:N"),
    opacity = alt.OpacityValue(op_var)
).add_params(
    op_var
)

alt.Chart(...)

### Click and highlight

Maybe we don't want to change the opacity for all points. We jsut want to select part of the data to be highlighted. There are three types of selections in the Altair: 

- selection_point: select a single discrete value, by default on click events. You can also select multiple discrete values. The first value is selected on mouse click and additional values toggled using shift-click.
- selection_interval: select a continuous range of values, initiated by mouse drag.

In [11]:
#selection = alt.selection_point()
#selection = alt.selection_point(empty = False)
#selection = alt.selection_point(on = 'mouseover')
#selection = alt.selection_interval()
selection = alt.selection_interval(encodings = ['x'])

alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.condition(selection, 'Origin:N', alt.value('grey')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
).add_params(
    selection
)

alt.Chart(...)

### Select across multiple panels

This approach becomes even more powerful when the selection behavior is tied across multiple views of the data within a compound chart. 

In [14]:
selection = alt.selection_interval(encodings = ['x'])

chart = alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.condition(selection, 'Origin:N', alt.value('grey')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
).add_params(
    selection
).properties(
    width = 200,
    height = 200
)

chart | chart.encode(x = "Acceleration:Q")

alt.HConcatChart(...)

Another thing we can do is to "zoom" the figure. We use the selection to get a filter of the data and make new figures. 

In [17]:
selection = alt.selection_interval(encodings = ['x'])

points = alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.condition(selection, 'Origin:N', alt.value('grey')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
).add_params(
    selection
)

bars = alt.Chart(cars).mark_bar().encode(
    x = 'count()', 
    y = 'Origin:N',
    color = 'Origin:N'
).transform_filter(
    selection
)

points & bars

alt.VConcatChart(...)

The interaction can also be added to the legend instead of the figure itself. In the following example, we can click the legend and only show part of the data in the figure. 

In [20]:
selection = alt.selection_point(fields = ['Origin', 'Cylinders'])

point = alt.Chart(cars).mark_circle().encode(
    x = alt.X('Horsepower:Q'),
    y = alt.Y('Miles_per_Gallon:Q'), 
    color = alt.condition(selection, alt.Color('Origin:N').legend(None), alt.value('grey')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

legend = alt.Chart(cars).mark_rect().encode(
    y = alt.Y('Origin:N').axis(orient = 'right'), 
    x = alt.X('Cylinders:O'),
    color = alt.condition(selection, alt.Color('Origin:N').legend(None), alt.value('grey'))
).add_params(
    selection
)

point | legend

alt.HConcatChart(...)

### Bindings & Widgets

We can now bind parameters to chart elements (e.g. legends) and widgets (e.g. drop-downs and sliders) to allow users to do detailed screening on the data. 

- binding_checkbox: Renders as checkboxes allowing for multiple selections of items.
- binding_radio: Radio buttons that force only a single selection
- binding_select: Drop down box for selecting a single item from a list
- binding_range: Shown as a slider to allow for selection along a scale.

[Examples](https://altair-viz.github.io/gallery/multiple_interactions.html#gallery-multiple-interactions)

What we need to setup: 
- Set the dropdown (options, name etc. )
- Use selection to combine dropdown to the figure
- Make the figure

In [23]:
input_dropdown = alt.binding_select(options = [None,'Europe', 'Japan', 'USA'],
                                   labels = ['All', 'Europe', 'Japan', 'USA'], 
                                   name = 'Region: ')

selection = alt.selection_point(fields = ['Origin'], bind = input_dropdown)

alt.Chart(cars).mark_circle().encode(
    x = 'Horsepower:Q',
    y = 'Miles_per_Gallon:Q', 
    color = alt.condition(selection, alt.Color('Origin:N'), alt.value('lightgrey'))
).add_params(
    selection
)

alt.Chart(...)

In [25]:
input_dropdown = alt.binding_radio(options = [None,'Europe', 'Japan', 'USA'],
                                   labels = ['All', 'Europe', 'Japan', 'USA'], 
                                   name = 'Region: ')

selection = alt.selection_point(fields = ['Origin'], bind = input_dropdown)

alt.Chart(cars).mark_circle().encode(
    x = 'Horsepower:Q',
    y = 'Miles_per_Gallon:Q', 
    color = alt.condition(selection, alt.Color('Origin:N'), alt.value('lightgrey'))
).add_params(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

In [27]:
# Replicate example 1
shape_select = alt.binding_select(options = ['arrow', 'circle', 'square', 'cross', 'triangle'], name = "Shape: ")
shape_var = alt.param(bind = shape_select, value = 'circle')

angle_slider = alt.binding_range(min = -360, max = 360, step = 1, name = 'Angle: ')
angle_var = alt.param(bind = angle_slider, value = 0)

size_slider = alt.binding_range(min = 10, max = 500, step = 10, name = 'Size: ')
size_var = alt.param(bind = size_slider, value = 50)

alt.Chart(cars).mark_point(
    shape = shape_var,
    angle = angle_var,
    size = size_var
).encode(
    x = 'Horsepower:Q', 
    y = 'Miles_per_Gallon:Q'
).add_params(shape_var, angle_var, size_var)

alt.Chart(...)

In [29]:
# Replicate example 2

slider = alt.binding_range(min = 0, max = 240, step = 1, name = "Cutoff: ")
selection = alt.param(bind = slider, value = 120)

alt.Chart(cars).mark_point().encode(
    x = 'Horsepower:Q', 
    y = 'Miles_per_Gallon:Q',
    color = alt.condition(
        alt.datum.Horsepower < selection, 
        alt.value('red'), 
        alt.value('blue')
    )
).add_params(selection)

alt.Chart(...)

### Exercise:

Create a scatter plot that shows the relationship between Horsepower and Acceleration. Color the points based on the Origin of the cars. To do:

- Add a dynamic filtering option that allows users to filter the dataset based on the number of cylinders. 
- Create a second plot: a bar chart showing the number of cars by the number of cylinders.
- Add an interactive slider that allows users to filter the data based on the car's weight

In [32]:
chart = alt.Chart(cars).mark_circle().encode(
    x = 'Horsepower:Q', 
    y = 'Acceleration:Q',
    color = 'Origin:N'
)
chart

alt.Chart(...)

In [34]:
radio = alt.binding_radio(options = [3,4,5,6,8], name = "Cylinders")
selection = alt.selection_point(fields = ['Cylinders'], bind = radio)

chart.add_params(selection).transform_filter(selection)

alt.Chart(...)

In [36]:
selection = alt.selection_interval()

points = chart.add_params(selection)

bars = alt.Chart(cars).mark_bar().encode(
    x = 'Cylinders:O', 
    y = 'count()'
).transform_filter(
    selection
)

points | bars

alt.HConcatChart(...)

In [38]:
weight_slider = alt.binding_range(min = 1500, max = 5000, step = 100, name = "Weight: ")
selection = alt.param(value = 3000, bind = weight_slider)

chart.add_params(selection).transform_filter(
    alt.datum.Weight_in_lbs <= selection
)

alt.Chart(...)

### In-class activities: 

Titanic data again!
Now, take the three Altair plots you have made last time. Select one of the plots and include some interactions to it (beyond the tooltips this time!). Save the plot as an html file. 

In [57]:
import altair as alt
import pandas as pd
import seaborn as sns

# Load Titanic dataset
titanic_data = sns.load_dataset('titanic')
print(titanic_data)

     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

In [59]:
# Convert categorical columns to string to avoid encoding issues
titanic_data['sex'] = titanic_data['sex'].astype(str)
titanic_data['survived'] = titanic_data['survived'].astype(str)

In [67]:
# Create an interactive selection
selection = alt.selection_single(
    fields=['survived'],
    bind=alt.binding_radio(name='Survived: ', options=['0', '1']),
    value='1'  # Corrected init argument
)

/var/folders/3h/2c5rpls550q5t_jwdflxbycm0000gn/T/ipykernel_13315/1910868002.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single(


In [73]:
# Interactive Countplot in Altair
interactive_count_chart = alt.Chart(titanic_data).mark_bar().encode(
    x=alt.X('sex:N', title='Gender'),
    y=alt.Y('count()', title='Count'),
    color=alt.Color('survived:N', title='Survived', scale=alt.Scale(domain=['0', '1'], range=['steelblue', 'orange'])),
    tooltip=['sex', 'count()', 'survived']
).add_selection(selection).transform_filter(selection).properties(
    title='Interactive Survival Count by Gender'
)


/var/folders/3h/2c5rpls550q5t_jwdflxbycm0000gn/T/ipykernel_13315/2911589295.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  interactive_count_chart = alt.Chart(titanic_data).mark_bar().encode(


In [75]:
# Save the interactive plot as an HTML file
interactive_count_chart.save('/Users/arushibhatia/Downloads/DS4200/interactive_titanic_plot.html')